## Задание 1 (yargy)

In [ ]:
!pip install yargy

In [ ]:
from yargy import Parser, rule, or_
from yargy.predicates import in_, in_caseless, custom
from yargy.tokenizer import MorphTokenizer
from yargy.pipelines import morph_pipeline, caseless_pipeline
from yargy.interpretation import fact

In [ ]:
import pandas as pd

In [ ]:
with open ('avito_cars.txt', 'r') as f:
  data = f.read()

In [ ]:
print(data[:100])

Кондиционер, гидроусилитель руля, электрические стеклоподъемники. Мотор как  на Opel. Дополнительно 


In [ ]:
Car = fact(
    'Car',
    ['brand', 'model', 'cross', 'year']
)

In [ ]:
vaz_models = ['2107', 'семёрка', 'семерка', 'ларгус', 'пятнашка', '2115', 'largus', 'kalina', 'priora', 'калина', 'приора' '21110', 'XRay', 'vesta', 'веста', 'гранта', 'granta', '2109', 'девятка']

In [ ]:
vaz_models_true_names = ['2107', '2107', '2107', 'ларгус', '2115', '2115', 'ларгус', 'калина', 'приора', 'калина', 'приора', '21110', 'XRAY', 'веста', 'веста', 'гранта', 'гранта', '2109', '2109']
vaz_models_dict = dict(zip(vaz_models, vaz_models_true_names))

In [ ]:
KEY = or_(
    rule('г', '.', 'в', '.'),
    rule('год'),
    rule('года'),
    rule('год', 'выпуска'),
    rule('года', 'выпуска')

).named('KEY')
predicate = custom(lambda x: str.isdigit(x))
VALUE = predicate.named('VALUE')
year = rule(
    KEY,
    VALUE
).named('year')
#YEAR.normalized.as_bnf

In [ ]:
vaz = rule(
    morph_pipeline(['ВАЗ', 'Лада', "Жигули", 'Lada']).interpretation(Car.brand.const('ВАЗ/Лада')),
    morph_pipeline(vaz_models_dict).interpretation(Car.model.normalized().custom(vaz_models_dict.get)),
    morph_pipeline(['Cross']).interpretation(Car.cross.normalized()).optional(),
    year.interpretation(Car.year.normalized()).optional()
    )
#.custom({'г.в.' : 'год выпуска', 'г.в.' : 'год', 'год выпуск' : 'год выпуска'}.get)

volvo = rule(
    morph_pipeline(['volvo', 'вольво']).interpretation(Car.brand.const('volvo')),
    morph_pipeline(['xc90', 'xc70', 's60', 'xc40', 'v60', 'v90', 's90', 's80', '740', '760', '940', '960', 'c30']).interpretation(Car.model),
    morph_pipeline(['Cross Country']).interpretation(Car.cross.normalized()).optional(),
    year.interpretation(Car.year.normalized()).optional()
    )
Car = or_(vaz, volvo).interpretation(Car)

parser = Parser(Car)

In [ ]:
%%time
matches_raw = list(parser.findall(data))

CPU times: user 2min 24s, sys: 6.61 s, total: 2min 31s
Wall time: 2min 31s


In [ ]:
matches = []
for match in matches_raw:
  matches.append(match.fact)

In [ ]:
 len(matches)

1366

In [ ]:
matches = set(matches)

In [ ]:
len(matches)

36

In [ ]:
i = 1
for match in matches:
    if match.cross == None:
      match.cross = ''
    if match.year == None:
      match.year = ''
    print(i, match.brand, match.model, match.cross, match.year)
    i += 1

1 ВАЗ/Лада 2115  год 2009
2 volvo S60  
3 ВАЗ/Лада веста  год выпуск 2017
4 ВАЗ/Лада калина  
5 volvo XC 70  
6 ВАЗ/Лада 2107  г.в.1986
7 volvo XC 90  
8 ВАЗ/Лада ларгус Cross 
9 ВАЗ/Лада 2107  год выпуск 2006
10 ВАЗ/Лада гранта  
11 volvo S80  
12 ВАЗ/Лада 2115  год выпуск 2009
13 ВАЗ/Лада XRAY  
14 ВАЗ/Лада 2107  год выпуск 2011
15 volvo 740  
16 ВАЗ/Лада приора  
17 volvo xc90  
18 ВАЗ/Лада 2107  год 2001
19 ВАЗ/Лада ларгус  год выпуск 2012
20 volvo XC90  
21 volvo s80  
22 ВАЗ/Лада 2115  год выпуск 2008
23 ВАЗ/Лада калина  год выпуск 2017
24 ВАЗ/Лада 2115  
25 volvo XC70  
26 volvo C30  
27 ВАЗ/Лада ларгус  
28 volvo s 80  
29 ВАЗ/Лада 2107  год 2005
30 ВАЗ/Лада 2107  
31 ВАЗ/Лада 2107  год 2004
32 ВАЗ/Лада веста  
33 ВАЗ/Лада 21110  
34 ВАЗ/Лада калина Cross 
35 volvo 940  
36 ВАЗ/Лада 2107  год выпуск 1985
